In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect

In [5]:
# Connect the engine to the database file we'll be using
engine = create_engine('sqlite:///chinook.db')
engine


Engine(sqlite:///chinook.db)

# Extract

In [6]:
# SQL Expression Language creates metadata that contains objects that define the customers table
metadata = MetaData()

# This method instantiates the tables that already 
# exist in the database, which the engine is connected to. 
metadata.create_all(engine)

# Checking this out, we can see the table structure and variable types for the employees table
inspector = inspect(engine)

# Transform

In [7]:
import pandas as pd
df = pd.read_sql_query("""SELECT i.InvoiceId,
       i.CustomerId,
       substr(i.InvoiceDate, 1, 10) AS InvoiceDate,
       i.BillingAddress,
       i.BillingCity,
       i.BillingCountry,
       count(it.trackId) AS Items,
       i.Total
  FROM invoices i
       INNER JOIN
       invoice_items it ON i.invoiceId = it.invoiceId
 GROUP BY 1;
    """, con=engine.connect())

In [8]:
df1 = pd.read_sql_query("""SELECT c.customerId,
       c.firstname,
       c.lastname,
       c.company,
       c.address,
       c.city,
       c.country,
       c.state,
       c.phone,
       c.email,
       c.supportrepid
  FROM customers c
 WHERE c.country LIKE 'USA';
    """, con=engine.connect())

In [9]:
df2 = pd.read_sql_query("""SELECT e.employeeid,
       e.firstname,
       e.lastname,
       e.title,
       e.reportsto,
       substr(e.BirthDate, 1, 10) AS BirthDate,
       substr(e.HireDate, 1, 10) AS BirthDate,
       e.address,
       e.city,
       e.country,
       e.PostalCode,
       e.phone,
       e.email
  FROM employees e;
    """, con=engine.connect())


In [10]:
df3 = pd.read_sql_query("""SELECT c.customerId as ID,
       c.FIRSTNAME || ' ' || c.LASTNAME AS Cliente,
       e.FIRSTNAME || ' ' || e.LASTNAME AS EmpleadoAsignado,
       count(DISTINCT g.Name) as 'Cantidad de Generos Comprados',
       count(DISTINCT al.Title) as 'Cantidad de Albums Comprados',
       sum(it.UnitPrice) AS Total
  FROM invoices i
       INNER JOIN
       customers c ON i.customerId = c.customerId
       JOIN
       employees e ON c.SupportRepId = e.EmployeeId
       JOIN
       invoice_items it ON i.invoiceId = it.invoiceId
       JOIN
       tracks t ON t.trackId = it.trackId
       INNER JOIN
       genres g ON t.genreId = g.genreId
       INNER JOIN
       albums al ON t.albumId = al.albumId
 GROUP BY 1
 ORDER BY 6 DESC;
    """, con=engine.connect())


In [22]:
df4 = pd.read_sql_query("""SELECT t.Name AS [Nombre canción],
       t.UnitPrice AS [Precio unitario],
       m.Name AS [Tipo de archivo],
       p.Name AS Playlist,
       ab.Title AS Album,
       art.Name AS Artista,
       g.Name AS Genero,
       count(v.InvoiceDate) AS VecesComprada
  FROM tracks t
       LEFT JOIN
       media_types mt ON mt.MediaTypeId = t.MediaTypeId
       LEFT JOIN
       media_types AS m ON t.MediaTypeId = m.MediaTypeId
       LEFT JOIN
       playlist_track AS pt ON pt.TrackId = t.TrackId
       LEFT JOIN
       playlists p ON p.PlaylistId = pt.PlaylistId
       LEFT JOIN
       albums ab ON ab.AlbumId = t.AlbumId
       LEFT JOIN
       artists art ON art.ArtistId = ab.ArtistId
       LEFT JOIN
       genres g ON g.GenreId = t.GenreId
       JOIN
       invoice_items vl ON vl.TrackId = t.TrackId
       JOIN
       invoices v ON v.InvoiceId = vl.InvoiceId
 GROUP BY 1
 ORDER BY 8 DESC;
    """, con=engine.connect())




In [12]:
df.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingCountry,Items,Total
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,Germany,2,1.98
1,2,4,2009-01-02,Ullevålsveien 14,Oslo,Norway,4,3.96
2,3,8,2009-01-03,Grétrystraat 63,Brussels,Belgium,6,5.94
3,4,14,2009-01-06,8210 111 ST NW,Edmonton,Canada,9,8.91
4,5,23,2009-01-11,69 Salem Street,Boston,USA,14,13.86


In [13]:
df1.head()

,CustomerId,FirstName,LastName,Company,Address,City,Country,State,Phone,Email,SupportRepId
0,16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,USA,CA,+1 (650) 253-0000,fharris@google.com,4
1,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,USA,WA,+1 (425) 882-8080,jacksmith@microsoft.com,5
2,18,Michelle,Brooks,None,627 Broadway,New York,USA,NY,+1 (212) 221-3546,michelleb@aol.com,3
3,19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,USA,CA,+1 (408) 996-1010,tgoyer@apple.com,3
4,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,USA,CA,+1 (650) 644-3358,dmiller@comcast.com,4


In [14]:
df2.head();

In [15]:
df3.head();

In [23]:
df4.head();

# Load


In [17]:
#Export dataframe
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

targetfile = "transformed_invoices.csv"
load(targetfile, df)

In [18]:
#Export dataframe
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

targetfile = "transformed_usa_customers.csv"
load(targetfile, df1)

In [19]:
#Export dataframe
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

targetfile = "transformed_employees.csv"
load(targetfile, df2)

In [20]:
#Export dataframe
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

targetfile = "transformed_customers_invoices.csv"
load(targetfile, df3)

In [24]:
#Export dataframe
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

targetfile = "transformed_tracks_count.csv"
load(targetfile, df4)